# HW3 Programming: Finding Optimal Policies with Dynamic Programming

Your assignment is to:
  * modify this notebook as specified to generate the figures 3.2 and 3.5 in our textbook
  

I've marked the code cells that you should *not* change with comments. Sections where you have something to do are marked "TODO."

You should start by duplicating this notebook:
  - To save to your Google Drive, choose _File/Save a Copy in Drive…_ (from there you can open the file in Colab)
  - To save to your local machine, choose _File/Download/Download .ipnb_ (from there you can open the file in your favorite editor for notebooks)

When you submit, you will download the _.ipnb_ file and then upload it to Gradescope (details below).





In [1]:
import math

Note that we don't import any modules for random number generation. These dynamic programming techniques make use of probability information, but don't make use of any randomly generated quantities.

In [2]:
# Do not change the contents of this cell!

GAMMA=0.9 # Throughout this notebook we will use a discount rate of 0.9

# MDP Class

To begin with, we'll define an abstract `MDP` class, along with a very simple subclass: `SimpleMDP`.

An `MDP` represents the environment. It's main job is to provide, for a given state and action pair, the probability distribution for next state and reward.

In [3]:
# Do not change the contents of this cell!

class MDP:
  """Represents an MDP (environment).  Has two instance variables: states and
  actions which are arrays of all possible states and actions."""
  def __init__(self, states, actions):
    self.states = states
    self.actions = actions

  def resultsFor(self, s, a):
    """Given a state and action, returns a dictionary keyed by (state, reward)
    pairs whose value is the probability of that pair.

    This is similar to the p(s', r | s, a) function."""
    raise NotImplementedError

  def formatv(self, v):
    """Returns an easy-to-read representation of the value function."""
    return str(v)

  def formatpi(self, pi):
    """Returns an easy-to-read representation of the policy."""
    return str(pi)

# SimpleMDP Class

This class is a concrete example of an `MDP` with 2 states and 2 actions.

In [4]:
# Do not change the contents of this cell!

class SimpleMDP(MDP):
  def __init__(self):
    super().__init__(["S1", "S2"], ["A1", "A2"])
    self.p = {
        ("S1", "A1"): {
            ("S1", 5): 0.25,    # p(S1, 5, S1, A1) = 0.25
            ("S2", 0): 0.75     # p(S2, 5, S1, A1) = 0.75
        },
        ("S1", "A2"): {
            ("S2", 0): 1,
        },
        ("S2", "A1"): {
            ("S2", 0): 1,
        },
        ("S2", "A2"): {
            ("S1", 3): 0.1,
            ("S2", 0): 0.9
        },
    }

  def resultsFor(self, s, a):
    """Given a state and action, returns a dictionary keyed by (state, reward)
    pairs whose value is the probability of that pair.

    This is similar to the p(s', r | s, a) function."""
    key = (s, a)
    return self.p[key]

  def __str__(self):
    return str(self.p)

We can test the `SimpleMDP` by returning a result for a given state and action as well as printing the valid states and actions.

We see that, given action `A1` from state `S1`, there's a 0.25 probability of moving to state `S1` and getting a reward of 5 and a 0.75 probability of moving to state `S2` and getting a reward of 0.

In [5]:
# Do not change the contents of this cell!

mdp = SimpleMDP()
print(f'actions: {mdp.actions}')
print(f'states: {mdp.states}')
print(f'resultsFor("S1", "A1"): {mdp.resultsFor("S1", "A1")}')

actions: ['A1', 'A2']
states: ['S1', 'S2']
resultsFor("S1", "A1"): {('S1', 5): 0.25, ('S2', 0): 0.75}


# Policy Class

Next, let's define an abstract class `Policy`. The main job of a policy is to give a probability distribution over actions to take in a given state.

In [6]:
# Do not change the contents of this cell!

class Policy:
  def __init__(self, mdp):
    self.mdp = mdp

  def actionsFor(self, state):
    """ Returns a dictionary keyed by actions, whose associated value
    is the probability of choosing that action.

    This is similar to the pi(a | s) function."""
    raise NotImplementedError

  def __str__(self):
    """Returns the desired action for each state."""
    return str({s: self.actionFor(s) for s in self.mdp.states})

# ConstantPolicy Class

This class is a simple example of a policy, specifically a policy that always takes the same action, no matter what state the agent is in.

In [7]:
# Do not change the contents of this cell!

class ConstantPolicy(Policy):
  def __init__(self, mdp, constantAction):
    super().__init__(mdp)
    self.constantAction = constantAction

  def actionsFor(self, state):
    return {self.constantAction: 1}


We can test the that the `ConstantPolicy` does what it is supposed to: always return the dictionary `{constantAction: 1}`, where `constantAction` is the action given to the constructor.

In [8]:
# Do not change the contents of this cell!

policy = ConstantPolicy(SimpleMDP(), "A1")
policy.actionsFor("S1")

{'A1': 1}

# TODO: EquiprobablePolicy Class

Now you'll implement a simple policy that chooses actions with equal probability.

The constructor takes an `MDP` object. You can use that to figure out what actions are possible (review the `MDP` class if you don't remember how). The `actionsFor` function should return a dictionary that contains every possible action, each with the probability 1/number of actions.

In [9]:
class EquiprobablePolicy(Policy):
  def __init__(self, mdp):
    super().__init__(mdp)

  def actionsFor(self, state):
    # return all the states each with a 1/n probability where n is the number of actions possible
    equal_prob = 1/len(mdp.actions)
    a = {}
    for action in mdp.actions:
      a[action] = equal_prob 
    return a

Now test your `EquiprobablePolicy` class. The following should print out the dictionary `{A1: 0.5, A2: 0.5}` (though the order may vary).

In [10]:
# Do not change the contents of this cell!

policy = EquiprobablePolicy(SimpleMDP())
policy.actionsFor("S1")

{'A1': 0.5, 'A2': 0.5}

# TODO: Iterative Policy Evaluation

Now, let's implement Iterative Policy Evaluation, which can be used to estimate the value function of a given policy in a given MDP.

In [11]:
def evaluatePolicy(mdp, pi, threshold=1e-4, v=None):
  """Take an MDP mdp, Policy pi and returns v, the approximation to v_pi
     (a dictionary keyed by states, with the output of v_pi(s) as values).

     The optional parameter threshold determines how
     accurate the approximation will be.

     The optional parameter v is an initial starting point
     (possibly better than the default, which always outputs 0)."""
  if v is None:
    curV = {s: 0 for s in mdp.states}
  else:
    curV = v.copy()

  delta = math.inf
  numIterations = 0
  while delta > threshold:
    numIterations += 1
    delta = 0
    for s in mdp.states:           # sweep over all the states
      oldValue = curV[s]           # remember the old value for comparison
      actions = pi.actionsFor(s)   # get the distribution over actions
      
      
      newVal = 0
      for action in actions:
        # In the MDP class the p attribute stores the p(s', r, | s, a) function
        # Format: {("s", "a") : {(s', r) : prob}}
        
        # getting pi(a | s)
        policy_prob = actions[action]

        # getting the next states and rewards associated with the current state and action
        # cur_p = mdp.p[(s, action)]  # current (s', r) : probability
        cur_p = mdp.resultsFor(s, action)

        for val in cur_p:
          s_prime, r = val
          sa_prob = cur_p[val]

          # increment sum
          newVal += policy_prob * sa_prob * (r + (GAMMA * curV[s_prime]))

      curV[s] = newVal 
          
      delta = max(delta, abs(oldValue - curV[s]))
  print("Evaluation Iterations: " + str(numIterations))
  # We've reached the accuracy threshold, so we can return
  return curV

Let's test your policy evaluation function on our `SimpleMDP`!

In [12]:
# Do not change the contents of this cell!

mdp = SimpleMDP()
pi = EquiprobablePolicy(mdp)
v = evaluatePolicy(mdp, pi)
print(mdp.formatv(v))

Evaluation Iterations: 70
{'S1': 2.237835524546828, 'S2': 1.7284565203872646}


You should get output that matches the following (though the order may vary):
```
{'S1': 2.237835524546828, 'S2': 1.7284565203872646}
```

# TODO: Gridworld

Now you'll implement the Gridworld environment in Example 3.5 of the book. You'll need to complete the `resultsFor` method which needs to give the probability distribution over new states and rewards, given the current state and action.

For this environment, each state will be a tuple `(x, y)` giving the agent's coordinates on the grid and the actions are the strings `"L"`, `"R"`, `"U"`, `"D"`, representing the four cardinal directions. Let the *top-left* corner represent state `(0, 0)`. Since the environment is deterministic, the returned dictionary will have only one entry.

In [13]:
# TODO: Fill in resultsFor

class GridWorld(MDP):
  def __init__(self):
    self.height=5
    self.width=5
    super().__init__([(x, y) for x in range(self.width) for y in range(self.height)],
                     ["L", "R", "U", "D"])

  def resultsFor(self, s, a):
    """Given a state and action, returns a dictionary keyed by state, reward
    pairs whose value is the probability of that pair.

    This is similar to the p(s', r | s, a) function."""

    x, y = s

    # Handle teleport states first
    if s == (1, 0):
        return {((1, 4), 10): 1}
    elif s == (3, 0):
        return {((3, 2), 5): 1}

    newX = x
    newY = y
    rwd = 0

    
    if a == "L":
      if x == 0:
        # left edge of board
        rwd = -1
      else:
        newX -= 1
    elif a == "R":
      if x == self.width-1:
        rwd = -1
      else:
        newX += 1
    elif a == "U":
      if y == 0:
        rwd = -1
      else: 
        newY -= 1
    elif a == "D":
      if y == self.height - 1:
        rwd = -1
      else:
        newY += 1


    # TODO: Correctly set the values of newX, newY, and rwd

    return {((newX, newY), rwd) : 1}

  def formatv(self, v):
    lines = []
    for y in range(self.height):
      lines.append(" ".join([f'{v[(x, y)]:4.1f}' for x in range(self.width)]))
    return "\n".join(lines)

  def formatpi(self, pi):
    lines = []
    for y in range(self.height):
      line = ""
      for x in range(self.width):
        acts = pi.actionsFor((x, y))
        if len(acts) == 1:
          line += list(acts.keys())[0] + " "
        else:
          for a in acts:
            line += a + "(" + str(acts[a]) + ")"
          line += " | "
      lines.append(line)
    return "\n".join(lines)

Now you can use all of your code together to generate the value function shown in Figure 3.2!

In [14]:
# Do not change the contents of this cell!

mdp = GridWorld()
pi = EquiprobablePolicy(mdp)
v = evaluatePolicy(mdp, pi)
print(mdp.formatv(v))

Evaluation Iterations: 43
 3.3  8.8  4.4  5.3  1.5
 1.5  3.0  2.3  1.9  0.5
 0.1  0.7  0.7  0.4 -0.4
-1.0 -0.4 -0.4 -0.6 -1.2
-1.9 -1.3 -1.2 -1.4 -2.0


The results should match those in the right-hand side of Figure 3.2:
```
 3.3  8.8  4.4  5.3  1.5
 1.5  3.0  2.3  1.9  0.5
 0.1  0.7  0.7  0.4 -0.4
-1.0 -0.4 -0.4 -0.6 -1.2
-1.9 -1.3 -1.2 -1.4 -2.0
```

# TODO: GreedyPolicy Class

In order to improve a policy, you'll need to implement a class to represent the _greedy policy_ with respect to a given value function.

Specifically, this policy is constructed from a value function $v$. In state $s$, it always chooses the action $a$ with the highest value of $q(s, a)$:

$$
\begin{aligned} \pi(s) &= \text{argmax}_a q(s, a) \\
&= \text{argmax}_a \mathbb{E}[R_t + \gamma v(S_t) | S_{t-1} = s, A_{t-1} = a]\\
&= \text{argmax}_a \sum_{s', r} P(s', r | s, a)\left[ r + \gamma v_{\pi}(s') \right] \end{aligned}
$$

In every state the greedy policy deterministically picks one action, so the dictionary returned by `actionsFor` should have only one entry that has probability 1.

In [15]:
class GreedyPolicy(Policy):
  """Policy that is greedy with respect to the given value function, v."""
  def __init__(self, mdp, v):
    super().__init__(mdp)
    self.v = v.copy()  # Copy so that changes to v don't affect us

  def actionsFor(self, s):
    maxAct = None
    maxQ = -math.inf

    # We need to calculate the Q value for each possible action given a state s
    # Use the bellman equation for action value

    # need to iterate through each possible action given our state s
    for action in mdp.actions:
      cur_q = 0
      p_function = mdp.resultsFor(s, action) # the p_function has {}

      # unpack s', r and the probability
      for key in p_function:
        s_prime, r = key
        prob = p_function[key]

        cur_q += prob * (r + (GAMMA * self.v[s_prime]))

      if cur_q > maxQ:
        maxAct = action # Updating the max action
      maxQ = max(maxQ, cur_q) # Updating the max Q

    return {maxAct : 1}

# TODO: Policy Iteration

Now we can use the `evaluatePolicy` function and the `GreedyPolicy` class within the Policy Iteration algorithm, which can find an optimal policy for our MDP!

Using these two components, complete the implementation of the `policyIteration` function below.

In [16]:
# TODO: Fill in missing code!

def policyIteration(mdp, pi, threshold=1e-4):
  """Takes an MDP mdp, initial policy pi and returns a tuple (vStar, piStar),
     respectively the approximately optimal value function, and an optimal policy.

     The optional parameter threshold determines how
     accurate each policy evaluation will be.
  """
  iteration = 0
  curPi = pi
  policyStable = False
  while not policyStable:
    print("Policy Iteration " + str(iteration))
    curV = evaluatePolicy(mdp, curPi, threshold)

    # Printing out current value function and policy
    # Don't change this!
    print("v")
    print(mdp.formatv(curV))
    print("\npi")
    print(mdp.formatpi(curPi) + "\n")
    iteration += 1

    # Create improved policy
    nextPi = GreedyPolicy(mdp, curV)

    policyStable = True
    for state in mdp.states:
      if curPi.actionsFor(state) != nextPi.actionsFor(state):
        policyStable = False
    curPi = nextPi
    
  piStar = curPi
  vStar = curV
  return (vStar, piStar)

The following code will test out your policy iteration algorithm on the Gridworld example. Note that the value function and policy will be printed out at each iteration. The last value function should match the one given in Example 3.8 (page 65):

```
22.0 24.4 22.0 19.4 17.5
19.8 22.0 19.8 17.8 16.0
17.8 19.8 17.8 16.0 14.4
16.0 17.8 16.0 14.4 13.0
14.4 16.0 14.4 13.0 11.7
```

The policy you get may not exactly match that figure (since our greedy policy is deterministic) but it should be _compatible_ with it (that is, the action it chooses should be one of the optimal possibilities shown in the figure).

In [17]:
# Do not change the contents of this cell!

mdp = GridWorld()
pi = EquiprobablePolicy(mdp)
vStar, piStar = policyIteration(mdp, pi)
print("-----------------\n")
print("(Approximate) v*")
print(mdp.formatv(vStar))
print("\npi*")
print(mdp.formatpi(piStar))

Policy Iteration 0
Evaluation Iterations: 43
v
 3.3  8.8  4.4  5.3  1.5
 1.5  3.0  2.3  1.9  0.5
 0.1  0.7  0.7  0.4 -0.4
-1.0 -0.4 -0.4 -0.6 -1.2
-1.9 -1.3 -1.2 -1.4 -2.0

pi
L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | 
L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | 
L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | 
L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | 
L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | L(0.25)R(0.25)U(0.25)D(0.25) | 

Policy Iteration 1
Evaluation Iterations: 3

# TODO: Value Iteration

One more thing! Now implement the Value Iteration algorithm by completing the `valueIteration` function below. It's very similar to the `evaluatePolicy` function, except that it uses the Bellman optimality equation to calculate the _optimal_ value function.

In [22]:
# TODO: Fill in missing code!

def valueIteration(mdp, threshold=1e-4, v=None):
  """Take an MDP mdp and returns a tuple (vStar, piStar),
     respectively the approximately optimal value function and policy.

     The optional parameter threshold determines how
     accurate the approximation will be.

     The optional parameter v is an initial starting point
     (possibly better than the default, which always outputs 0)."""
  if v is None:
    curV = {s: 0 for s in mdp.states}
  else:
    curV = v.copy()

  numIterations = 0
  delta = math.inf
  while delta > threshold:
    numIterations += 1
    delta = 0
    for s in mdp.states:           # sweep over all the states
      oldValue = curV[s]           # remember the old value for comparison
      actions = mdp.actions        # get the actions (no probabilities needed!)

      # TODO: Calculate the new value of curV[s]
      newValue = -math.inf
      for action in actions:
        q=0
        p_function = mdp.resultsFor(s, action)
        for key in p_function:
          s_prime, r = key
          prob = p_function[key]
          q += prob * (r + (GAMMA * curV[s_prime]))
        newValue = max(newValue, q)
      curV[s] = newValue


      delta = max(delta, abs(oldValue - curV[s]))
  print("Value Iterations: " + str(numIterations))
  # We've reached the accuracy threshold, so we can return
  return (curV, GreedyPolicy(mdp, curV))

The following code will test out your policy iteration algorithm on the Gridworld example. The value function should match the one given in Example 3.8 (page 65):

```
22.0 24.4 22.0 19.4 17.5
19.8 22.0 19.8 17.8 16.0
17.8 19.8 17.8 16.0 14.4
16.0 17.8 16.0 14.4 13.0
14.4 16.0 14.4 13.0 11.7
```

The policy you get may not exactly match that figure (since our greedy policy is deterministic) but it should be _compatible_ with it (that is, the action it chooses should be one of the optimal possibilities shown in the figure).

In [23]:
# Do not change the contents of this cell!

mdp = GridWorld()
pi = EquiprobablePolicy(mdp)
vStar, piStar = valueIteration(mdp)
print("----------------")
print("(Approximate) v*")
print(mdp.formatv(vStar))
print("\npi*")
print(mdp.formatpi(piStar))

Value Iterations: 24
----------------
(Approximate) v*
22.0 24.4 22.0 19.4 17.5
19.8 22.0 19.8 17.8 16.0
17.8 19.8 17.8 16.0 14.4
16.0 17.8 16.0 14.4 13.0
14.4 16.0 14.4 13.0 11.7

pi*
R L L L L 
R U L L L 
R U L L L 
R U L L L 
R U L L L 


# TODO: Submit!

1. Make sure that the output of all cells is up-to-date (easiest way is to choose _Runtime/Run all_).
2. If you are working in Colab, choose _File/Download/Download .ipnb_ (otherwise locate your _.ipnb_ file)
3. Go to the [Gradescope assignment](https://www.gradescope.com/courses/435656/assignments/2230644/) to submit.
4. Choose _Upload_ as the submission method and upload your _.ipnb_ file.
5. If you are working as a pair, make sure to push the _Group Members_ button to add the other partner.